## Important, graphite chip available. 🥶

In [ ]:
import torch
print(torch.backends.mps.is_available())

True


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Salesforce/codegen-350M-mono"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

In [10]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["qkv_proj", "out_proj", "fc_in", "fc_out"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [11]:
from datasets import Dataset

data = {
    "instruction": [
        "Crea un componente funcional de React que muestre 'Hola Mundo'.",
        "Haz un hook de React que maneje un contador."
    ],
    "output": [
        "import React from 'react';\n\nexport default function App() {\n  return <h1>Hola Mundo</h1>;\n}",
        "import { useState } from 'react';\n\nexport function useCounter() {\n  const [count, setCount] = useState(0);\n  const increment = () => setCount(c => c + 1);\n  return { count, increment };\n}"
    ]
}

dataset = Dataset.from_dict(data)


In [17]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

def tokenize(example):
    return tokenizer(
        example["instruction"] + "\n" + example["output"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

tokenizer.pad_token = tokenizer.eos_token
tokenized_dataset = dataset.map(tokenize)

training_args = TrainingArguments(
    output_dir="./lora-codegen-demo",
    per_device_train_batch_size=1,
    num_train_epochs=1,
    logging_steps=1,
    save_steps=5,
    learning_rate=2e-4,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

trainer.train()

Map: 100%|██████████| 2/2 [00:00<00:00, 278.47 examples/s]
/Users/juandiego/Documents/study/tensorflow/train_model_react_with_LoRA/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
1,2.517100
2,3.084700


TrainOutput(global_step=2, training_loss=2.8008837699890137, metrics={'train_runtime': 3.6507, 'train_samples_per_second': 0.548, 'train_steps_per_second': 0.548, 'total_flos': 942812430336.0, 'train_loss': 2.8008837699890137, 'epoch': 1.0})

In [18]:
tokenizer.pad_token = tokenizer.eos_token

In [24]:
from peft import PeftModel
base_model = AutoModelForCausalLM.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model, "./lora-codegen-demo/checkpoint-2")

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

In [25]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.backends.mps.is_available() else -1
)

Device set to use mps:0


In [26]:
prompt = "Crea un componente funcional en React que muestre 'Hola React' en pantalla."
outputs = generator(
    prompt,
    max_new_tokens=120,
    do_sample=True,
    top_p=0.9,
    temperature=0.7
)

print(outputs[0]["generated_text"])

Crea un componente funcional en React que muestre 'Hola React' en pantalla.
        #    
        #    - El componente en el React es un elemento de la clase Component, y una funcion.
        #      La funcion es el código que se le asigna a un componente.
        #      
        #      La funcion es llamada por el metodo render, y es la que llamará a los componentes de la clase Component.
        #    
        #    - Los argumentos de la funcion son:
        #      
        #      -


In [27]:
base_model = AutoModelForCausalLM.from_pretrained(model_name)
base_generator = pipeline("text-generation", model=base_model, tokenizer=tokenizer)

print(base_generator(prompt, max_new_tokens=120)[0]["generated_text"])


Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

Crea un componente funcional en React que muestre 'Hola React' en pantalla.
    """
    def __init__(self, name, style={"background-color: #FF66FF"}):
        self.name = name
        self.style = style


@app.callback(
    Output("screen", "figure"),
    [Input("button_1", "n_clicks"), Input("button_2", "n_clicks")]
)
def build_graph(n_clicks_1, n_clicks_2):
    """Ejecuta n_clicks_1
